<a href="https://colab.research.google.com/github/shadi159/Cloud-Computing/blob/main/Copy_of_cloudSping2025tut6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup
def fetch_page(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
            "Mozilla/5.0"
        )
    }
    try:
        response = requests.get(url, headers=headers)
        print("HTTP status:", response.status_code)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            return soup
        else:
            print("Response head:", response.text[:200])
            return None
    except Exception as e:
        print("Request error:", e)
        return None

In [ ]:
import re
def index_words(soup):
  index = {}
  words = re.findall(r'\w+', soup.get_text())
  for word in words:
    word = word.lower()
    if word in index:
      index[word] += 1
    else:
      index[word] = 1
  return index

In [ ]:
def remove_stop_words(index):
  stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
  for stop_word in stop_words:
    if stop_word in index:
      del index[stop_word]
  return index

In [ ]:
from nltk.stem import PorterStemmer
def apply_stemming(index):
  stemmer = PorterStemmer()
  stemmed_index = {}
  for word, count in index.items():
    stemmed_word = stemmer.stem(word)
    if stemmed_word in stemmed_index:
      stemmed_index[stemmed_word] += count
    else:
      stemmed_index[stemmed_word] = count
  return stemmed_index

In [ ]:
def search(query, index):
  query_words = re.findall(r'\w+', query.lower())
  results = {}
  for word in query_words:
    if word in index:
      results[word] = index[word]
  return results

In [ ]:
def search_engine(url, query):
  soup = fetch_page(url)
  if soup is None:
     return None
  index = index_words(soup)
  index = remove_stop_words(index)
  index = apply_stemming(index)
  results = search(query, index)
  return results

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'bird'
results = search_engine(url, query)
print(results)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'birds wings'
results = search_engine(url, query)
print(results)

In [ ]:
def search(query, index):
  stemmer = PorterStemmer()
  query_words = re.findall(r'\w+', query.lower())
  results = {}
  for word in query_words:
    word = stemmer.stem(word)
    if word in index:
      results[word] = index[word]
  return results

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'birds wings'
results = search_engine(url, query)
print(results)

In [ ]:
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank


In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'birds wings'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
   rank = rank*1/count
rank = 1-rank
print(rank)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'collage students'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)


In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'owls'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'Industry'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)



```
# This is formatted as code
```

### **מנוע המיועד למספר דפים**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
class WikiSearchEngine:
  def __init__(self):
    """Initialize the search engine"""
    self.base_url = "https://en.wikipedia.org/w/api.php"
    self.pages = []
    self.word_locations = defaultdict(list) # word -> [(page_id, frequency), ...]
    self.stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with'}
    return False
  def fetch_wiki_pages(self, topic, num_pages=5):
    """Fetch Wikipedia pages for given topic"""
    search_params = {
      "action": "query",
      "format": "json",
      "list": "search",
      "srsearch": topic,
      "srlimit": num_pages
    }
    try:
      response = requests.get(self.base_url, params=search_params)
      search_results = response.json()['query']['search']

      for result in search_results:
        content_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts|info",
            "pageids": result['pageid'],
            "inprop": "url",
            "explaintext": True
        }
        content_response = requests.get(self.base_url, params=content_params)
        page_data = content_response.json()['query']['pages'][str(result['pageid'])]
        self.pages.append({
          'id': result['pageid'],
          'title': page_data['title'],
          'url': page_data.get('fullurl', f"https://en.wikipedia.org/?curid={result['pageid']}"),
          'content': page_data['extract']
        })
      print(f"Retrieved: {page_data['title']}")
      return True

    except Exception as e:
      print(f"Error fetching pages: {str(e)}")

  def build_index(self):
        """Build a simple word location index"""
        self.word_locations.clear()

        # Process each page
        for page in self.pages:
            # Get all words from content
            words = re.findall(r'\w+', page['content'].lower())

            # Count word frequencies
            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            # Add to index with page information
            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

  def search(self, query, num_results=5):
        """Search pages using simple word frequency ranking.
        Ranks pages based on:1. Number of query words found in the page
        2. Total frequency of query words  """
        # Get query words
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                    if word.lower() not in self.stop_words]
        if not query_words:
            return []

        # Calculate scores for each page
        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        # For each query word
        for word in query_words:
            # Find pages containing this word
            for page_id, freq in self.word_locations.get(word, []):
                page_scores[page_id]['matches'] += 1
                page_scores[page_id]['total_freq'] += freq


        # Convert to list and sort
        ranked_results = [
            (page_id, scores['matches'], scores['total_freq'])
            for page_id, scores in page_scores.items()
        ]
        # Sort by number of matching words first, then by total frequency
        ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)
        # Format results
        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next(p for p in self.pages if p['id'] == page_id)
            # Find the first matching word context
            context = self.get_context(page['content'], query_words)
            results.append({
                'title': page['title'],
                'url': page['url'],
                'matching_words': matches,
                'total_frequency': total_freq,
                'context': context
            })
        return results


In [ ]:
def fetch_wiki_pages(self, topic, num_pages=5):
  """Fetch Wikipedia pages for given topic"""
  search_params = {
  "action": "query",
  "format": "json",
  "list": "search",
  "srsearch": topic,
  "srlimit": num_pages
  }
  try:
      response = requests.get(self.base_url, params=search_params)
      search_results = response.json()['query']['search']

      for result in search_results:
        content_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts|info",
            "pageids": result['pageid'],
            "inprop": "url",
            "explaintext": True
        }
        content_response = requests.get(self.base_url, params=content_params)
        page_data = content_response.json()['query']['pages'][str(result['pageid'])]
        self.pages.append({
          'id': result['pageid'],
          'title': page_data['title'],
          'url': page_data.get('fullurl', f"https://en.wikipedia.org/?curid={result['pageid']}"),
          'content': page_data['extract']
        })
        print(f"Retrieved: {page_data['title']}")
      return True

  except Exception as e:
    print(f"Error fetching pages: {str(e)}")


In [ ]:
    def build_index(self):
        """Build a simple word location index"""
        self.word_locations.clear()

        # Process each page
        for page in self.pages:
            # Get all words from content
            words = re.findall(r'\w+', page['content'].lower())

            # Count word frequencies
            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            # Add to index with page information
            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

    def search(self, query, num_results=5):
        """Search pages using simple word frequency ranking.
        Ranks pages based on:1. Number of query words found in the page
        2. Total frequency of query words  """
        # Get query words
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                      if word.lower() not in self.stop_words]
        if not query_words:
            return []

        # Calculate scores for each page
        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        # For each query word
        for word in query_words:
            # Find pages containing this word
            for page_id, freq in self.word_locations.get(word, []):
                page_scores[page_id]['matches'] += 1
                page_scores[page_id]['total_freq'] += freq


        # Convert to list and sort
        ranked_results = [
            (page_id, scores['matches'], scores['total_freq'])
            for page_id, scores in page_scores.items()
        ]
        # Sort by number of matching words first, then by total frequency
        ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)
        # Format results
        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next(p for p in self.pages if p['id'] == page_id)
            # Find the first matching word context
            context = self.get_context(page['content'], query_words)
            results.append({
                'title': page['title'],
                'url': page['url'],
                'matching_words': matches,
                'total_frequency': total_freq,
                'context': context
            })
        return results

**Our Independent Work**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict


class WebSearchEngine:
    def __init__(self):
        """Simple search engine over arbitrary web pages."""
        self.pages = []
        self.word_locations = defaultdict(list)
        self.stop_words = {
            'a', 'an', 'the', 'and', 'or', 'in', 'on',
            'at', 'to', 'for', 'of', 'with', 'is', 'are', 'this', 'that',
            'as', 'by', 'from', 'it', 'be', 'was', 'were', 'has', 'have'
        }

    def _extract_text_from_html(self, html):
        soup = BeautifulSoup(html, "html.parser")

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def add_page_from_url(self, url, page_id=None):
        try:
            print(f"Fetching: {url}")
            resp = requests.get(url, timeout=15)
            resp.raise_for_status()

            html = resp.text
            content = self._extract_text_from_html(html)

            soup = BeautifulSoup(html, "html.parser")
            title_tag = soup.find("title")
            title = title_tag.get_text(strip=True) if title_tag else url

            if page_id is None:
                page_id = len(self.pages) + 1

            self.pages.append({
                'id': page_id,
                'title': title,
                'url': url,
                'content': content
            })
            print(f"Added page: {title}")
            return True

        except Exception as e:
            print(f"Error fetching {url}: {e}")
            return False

    def add_pages_from_urls(self, urls):
        for i, url in enumerate(urls, start=1):
            self.add_page_from_url(url, page_id=i)

    def build_index(self):
        self.word_locations.clear()

        for page in self.pages:
            words = re.findall(r'\w+', page['content'].lower())

            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

    def get_context(self, text, query_words, window=40):
        text_lower = text.lower()
        for qw in query_words:
            idx = text_lower.find(qw)
            if idx != -1:
                start = max(0, idx - window)
                end = min(len(text), idx + len(qw) + window)
                snippet = text[start:end]
                return " ".join(snippet.split())
        return " ".join(text[:2 * window].split())

    def search(self, query, num_results=5):
        query_words = [
            w.lower()
            for w in re.findall(r'\w+', query)
            if w.lower() not in self.stop_words
        ]
        if not query_words:
            return []

        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        for word in query_words:
            for page_id, freq in self.word_locations.get(word, []):
                page_scores[page_id]['matches'] += 1
                page_scores[page_id]['total_freq'] += freq

        ranked = [
            (pid, info['matches'], info['total_freq'])
            for pid, info in page_scores.items()
        ]
        ranked.sort(key=lambda x: (x[1], x[2]), reverse=True)

        results = []
        for page_id, matches, total_freq in ranked[:num_results]:
            page = next(p for p in self.pages if p['id'] == page_id)
            context = self.get_context(page['content'], query_words)
            results.append({
                'title': page['title'],
                'url': page['url'],
                'matching_words': matches,
                'total_frequency': total_freq,
                'context': context
            })
        return results


In [ ]:
import requests

urls = [
    "https://link.springer.com/article/10.1007/s10462-024-11100-x",
    "https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.01419/full",
    "https://www.nature.com/articles/s41598-025-05102-0",
    "https://www.planthealthcentre.scot/plant-diseases/resources"
]

KEYWORDS = [
    "plant",
    "disease",
    "leaf",
    "detection",
    "color",
    "Vector",
    "Wilt",
    "Blight",
    "Lesion",
    "Identifying"
]

engine = WebSearchEngine()

engine.add_pages_from_urls(urls)

engine.build_index()

index_data = {
    "pages": {},
    "words": {}
}

for page in engine.pages:
    index_data["pages"][str(page["id"])] = {
        "title": page["title"],
        "url": page["url"]
    }

for kw in KEYWORDS:
    w = kw.lower()
    word_info = {}
    for page_id, freq in engine.word_locations.get(w, []):
        word_info[str(page_id)] = freq

    index_data["words"][w] = word_info

print(index_data)


In [ ]:
FIREBASE_URL = "https://identifying-plant-diseas-ef19d-default-rtdb.firebaseio.com/"

def save_index_to_firebase(index_data, path="plant_disease_index"):
    url = f"{FIREBASE_URL}/{path}.json"

    try:
        response = requests.put(url, json=index_data)
        response.raise_for_status()
        print("Saved to Firebase successfully!")
        print("Response:", response.json())
    except Exception as e:
        print("Error saving to Firebase:", e)

save_index_to_firebase(index_data)


In [ ]:
import matplotlib.pyplot as plt

def get_keyword_freq_for_page(engine, page_id, keywords):
    freqs = []
    for kw in keywords:
        kw = kw.lower()
        freq = 0
        for pid, f in engine.word_locations.get(kw, []):
            if pid == page_id:
                freq = f
                break
        freqs.append(freq)
    return freqs

for page in engine.pages:
    page_id = page["id"]
    title = page["title"]

    freqs = get_keyword_freq_for_page(engine, page_id, KEYWORDS)

    plt.figure()
    plt.bar(KEYWORDS, freqs)
    plt.title(f"Keyword frequencies in page: {title}")
    plt.xlabel("Keyword")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
